# Story to Image promt generator

## Installing Dependencies

In [7]:
%pip install crewai
%pip install ChatGoogleGenerativeAI
%pip install -q -U google-generativeai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement ChatGoogleGenerativeAI (from versions: none)
ERROR: No matching distribution found for ChatGoogleGenerativeAI


Note: you may need to restart the kernel to use updated packages.


## Importing

In [8]:
import os
from crewai import Agent, Task, Process, Crew
from langchain_google_genai import ChatGoogleGenerativeAI
from textwrap import dedent

## Configuring The LLM

In [9]:
GOOGLE_API_KEY = "****************************************"

api_gemini = os.environ.get(GOOGLE_API_KEY)
llm = ChatGoogleGenerativeAI(
    model="gemini-pro", verbose=True, temperature=0.1, google_api_key=GOOGLE_API_KEY
)

## Variables

Reading the story from 'Story Script file'

In [10]:
story = """"""
filename = "Story Scirpt.txt"
with open(filename, "r") as file:
  story = file.read()


## Agents

### 1. Character and Entity Identifier (Character Designer)
- **Role:**  Information Extractor
- **Goal:** Analyze the literary text to identify key characters, entities, and their descriptive details relevant for visual representation. This includes characters, locations, objects, and any other important elements mentioned in the story.
- **Backstory:**  This agent is a skilled information extractor, trained on vast amounts of textual data.  It excels at identifying named entities from the text and extracting relevant details about them.  These details form the foundation for creating visual representations of the characters and entities.
- **Key Capabilities:**
- Identify named entities (characters, locations, objects)
- Extract relevant descriptive details about identified entities

### 2. Scene Summarizer (Storyboard Artist)
- **Role:** Narrative Flow Analyzer
- **Goal:** Break down the story into key scenes in sequential order, extracting essential elements for image generation. This includes identifying pivotal moments, character interactions, and settings within each scene.
- **Backstory:**  A master of narrative flow, this agent is adept at identifying and summarizing the most important scenes within a story. It can condense complex narratives into their core elements, focusing on pivotal moments, character interactions, and settings. This ensures the generated visuals accurately capture the essence of the story. 
- **Key Capabilities:**
- Identify key scenes in the story
- Summarize each scene, capturing pivotal moments, character interactions, and settings

### 3. Image Prompt Generator
- **Role:**  Creative Storyteller
- **Goal:** Craft comprehensive prompts that translate identified elements and scene summaries into detailed instructions for image generation.
- **Backstory:**  This creative storyteller translates written descriptions into visual language. It utilizes the information provided by the Character and Entity Identifier and Scene Summarizer to craft comprehensive prompts that capture the essence of characters, scenes, and desired visual styles. These prompts act as blueprints for the image generation process.
- **Key Capabilities:**
- Utilize information about characters, entities, and scenes
- Craft comprehensive image prompts with details on characters, setting, and style

In [18]:
# CREW: Character and Entity Identifier
identifier = Agent(
    role="Character and Concept Designer",
    goal="Analyze the literary text to identify key characters, entities, and their descriptive details",
    backstory="""You are a skilled information extractor, trained on vast amounts of textual data. You excel at identifying named entities, such as characters, locations, and objects, and extracting relevant details about them from the text. Your keen eye helps capture crucial information that forms the foundation for visual representation.""",
    verbose=True,
    allow_delegation=True,
      llm=llm 
)

# CREW: Scene Summarizer
summarizer = Agent(
    role="Storyboard Artist",
    goal="Break down the story into key scenes in sequential order, extracting essential elements for image generation",
    backstory="""You are a master of narrative flow, adept at identifying and summarizing the most important scenes within a story. You can condense complex narratives into their core elements, focusing on pivotal moments, character interactions, and settings. This helps ensure the generated visuals accurately capture the essence of the story.""",
    verbose=True,
    allow_delegation=False,
      llm=llm 
)

# CREW: Image Prompt Generator
prompter = Agent(
    role="Image Prompt Generator",
    goal="Craft comprehensive prompts that translate identified elements into detailed instructions for image generation",
    backstory="""You are a creative storyteller with a knack for translating written descriptions into visual language. You utilize the information provided by the Character & Entity Identifier and Scene Summarizer to craft comprehensive prompts that capture the essence of characters, scenes, and desired visual styles. These prompts act as blueprints for the image generation process.""",
    verbose=True,
    allow_delegation=False,
      llm=llm 
)

## Tasks

**1. Character and Entity Identification (identify_elements Task)**

**Description:**

This task analyzes the provided literary text to identify key characters, entities, and their descriptive details relevant for visual representation. This includes characters, locations, objects, and any other important elements mentioned in the story. 

**Input:**

* The literary text story (variable `story`).

**Expected Output:**

* A comprehensive list of identified characters, entities, and their descriptions. These descriptions should include details that can be visually represented, such as appearance, personality traits, surroundings, or other relevant details.

**2. Scene Summarization (summarize_scenes Task)**

**Description:**

This task breaks down the story into key scenes, extracting essential elements for image generation. It identifies pivotal moments, character interactions, and settings within each scene.

**Input:**

* The literary text story (variable `story`).

**Expected Output:**

* A list of scenes, each with a concise summary capturing the most important elements that contribute to the visual narrative. These summaries should highlight pivotal moments, character interactions, and settings relevant for generating images.

**3. Image Prompt Generation (generate_prompts Task)**

**Description:**

This task crafts comprehensive image prompts that translate the identified elements and scene summaries into detailed instructions for image generation. It utilizes the information from the Character and Entity Identifier and Scene Summarizer tasks to create prompts that capture the essence of characters, scenes, and desired visual styles.

**Input:**

* The output from the `identify_elements` task (list of characters and descriptions).
* The output from the `summarize_scenes` task (list of scene summaries).

**Expected Output:**

* A set of image prompts, one for each key scene in the story. Each prompt should comprehensively describe the scene, including characters, setting, atmosphere, and desired visual style.


In [19]:
# Character and entities archive
identify_elements = Task(
    description=dedent(f"""
        Analyze the provided literary text story to identify key characters, entities, 
        and their descriptive details relevant to visual representation. 
        This includes characters, locations, objects, and any other 
        important elements mentioned in the story.

        Your output should be a comprehensive list of identified 
        elements with their descriptions, providing the foundation for 
        further processing.
   
        **Story:**
         """ + story),
    agent=identifier,
    expected_output=dedent("""
        A list of identified characters, entities, and their descriptions, 
        including their appearance, personality traits, surroundings, or 
        any other relevant details that can be visually represented.
    """),
    async_execution=False
)

# scenes list
summarize_scenes = Task(
    description=dedent("""
        Break down the story into key scenes, extracting essential elements for image generation. Identify the pivotal moments, character interactions, and settings within each scene.

        Your output should be a list of scenes, each with a concise summary capturing the most important elements that contribute to the visual narrative.

        **Story:**
         """ + story),
    agent=summarizer,
    expected_output=dedent("""
        Your output should be a list of scenes, divided into key scenes, with each providing a concise summary capturing the most important elements that contribute to the visual narrative. This includes highlighting pivotal moments, character interactions, and settings relevant for image generation.
    """),
    async_execution=False
)

# Image Prompts
generate_prompts = Task(
    description=dedent(f"""
        Craft comprehensive image prompts that translate the identified 
        elements and scene summaries into detailed instructions for 
        image generation. Utilize the information from the Character & 
        Entity Identifier and Scene Summarizer to create prompts that capture the essence 
        of characters, scenes, and desired visual styles.

        Your output should be a set of image prompts, each corresponding 
        to a key scene in the story, providing clear and comprehensive 
        instructions for the image generation process.
    """),
    agent=prompter,
    expected_output=("""
        A set of image prompts, one for each key scene in the story. 
        Each prompt should comprehensively describe the scene, including 
        characters, setting, atmosphere, and desired visual style.
    """),
    async_execution=False,
    context=[identify_elements, summarize_scenes]
)

## Output by running the crew
Input your story in the 'Story Scirpt.txt' file

In [20]:
crew = Crew(
    agents=[ identifier, summarizer, prompter ],
    tasks=[ identify_elements, summarize_scenes, generate_prompts ],
    verbose=2,
    process=Process.sequential,  
)

result = crew.kickoff()

print("\n\n||||||||||||||||||||||||||||||||||||||||||||||||||||||||||\n\n")
print(result)

[DEBUG]: Working Agent: Character and Concept Designer
[INFO]: Starting Task: 
        Analyze the provided literary text story to identify key characters, entities, 
        and their descriptive details relevant to visual representation. 
        This includes characters, locations, objects, and any other 
        important elements mentioned in the story.

        Your output should be a comprehensive list of identified 
        elements with their descriptions, providing the foundation for 
        further processing.

        **Story:**
         A Midsummer Nightâ€™s Dream
There was a law in the city of Athens which gave to its citizens the power of compelling their daughters to marry whomsoever they pleased; for upon a daughterâ€™s refusing to marry the man her father had chosen to be her husband, the father was empowered by this law to cause her to be put to death; but as fathers do not often desire the death of their own daughters, even though they do happen to prove a little r

## Saving output in .txt file

In [21]:
with open("result.txt", "w") as file:
    file.write(result + "\n")
print("Text saved to result.txt successfully!")


Text saved to result.txt successfully!
